# NC Elections Results 2024

This processes election results for North Carolina's legislative elections of November 2024. It's built on data released right after the election so that it's not yet official:

https://www.ncsbe.gov/results-data/election-results/historical-election-results-data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('results_pct_20241105.txt', sep='\t')
data = data[data['Contest Type'] == 'S']
legislature = data[data['Contest Name'].apply(lambda s: len(s) > 14 and (s[:8] == "US HOUSE" or s[:8] == "NC HOUSE" or s[:15] == "NC STATE SENATE"))]
legislature.head()

,County,Election Date,Precinct,Contest Group ID,Contest Type,Contest Name,Choice,Choice Party,Vote For,Election Day,Early Voting,Absentee by Mail,Provisional,Total Votes,Real Precinct
14,BEAUFORT,11/05/2024,CHOCO,1259,S,NC HOUSE OF REPRESENTATIVES DISTRICT 079,Keith Kidwell,REP,1,309,0,0,0,309,Y
18,BEAUFORT,11/05/2024,EV-AUR,1381,S,US HOUSE OF REPRESENTATIVES DISTRICT 03,Greg Murphy,REP,1,0,380,0,0,380,N
30,BEAUFORT,11/05/2024,HB,1381,S,US HOUSE OF REPRESENTATIVES DISTRICT 03,Greg Murphy,REP,1,232,0,0,0,232,Y
34,BEAUFORT,11/05/2024,NCRK,1381,S,US HOUSE OF REPRESENTATIVES DISTRICT 03,Greg Murphy,REP,1,312,0,0,0,312,Y
38,BEAUFORT,11/05/2024,PANTE,1381,S,US HOUSE OF REPRESENTATIVES DISTRICT 03,Greg Murphy,REP,1,247,0,0,0,247,Y


In [23]:
# Magical Pandas part
results = legislature.groupby(
    ['Contest Name', 'Choice Party']
).agg(
    name=('Choice', 'first'), 
    votes=('Total Votes', 'sum')
).reset_index()

# Separate out the chamber and district values from the contest
def separate_contest(contest):
    if contest[:8] == "NC HOUSE":
        return ["nc_house", int(contest[-3:])]
    elif contest[:8] == "US HOUSE":
        return ["us_house", int(contest[-2:])]
    elif contest[:15] == "NC STATE SENATE":
        return ["nc_senate", int(contest[-2:])]
chamber_district = results['Contest Name'].apply(separate_contest)

# Reformat using simple keys
results['chamber'] = [cd[0] for cd in chamber_district]
results['district'] = [cd[1] for cd in chamber_district]
results['party'] = results['Choice Party'].apply(lambda s: s.lower())
results = results[['chamber', 'district', 'party', 'name', 'votes']]

results

,chamber,district,party,name,votes
0,nc_house,1,dem,Susan A. Sawin,17048
1,nc_house,1,rep,Edward C. Goodwin,31789
2,nc_house,2,dem,Ray Jeffers,28071
3,nc_house,2,rep,Jason Chambers,20775
4,nc_house,3,dem,Linda G. Moore,18321
...,...,...,...,...,...
362,us_house,12,rep,Addul Ali,90230
363,us_house,13,dem,Frank Pierce,170161
364,us_house,13,rep,Brad Knott,242208
365,us_house,14,dem,Pam Genant,166770


In [22]:
results.to_csv('nc_legislative_results_2024.csv', index=False)